In [ ]:
import tensorflow as tf
import numpy as np

print(f"tf version {tf.__version__}")
print(f"Numpy version {np.__version__}")

In [13]:
BATCH_SIZE = 64 
EPOCHS = 5
class Encoder(tf.keras.layers.Layer):
	"""
	Encoder Layer , To Learned the features of input data and  extract the important vfeatures

	Args:
	    tf (Layer): Model Subclassing 
	"""
	def __init__(self, Intermediate_dims):
		super(Encoder, self).__init__()
		
		self.hidden_layer = tf.keras.layers.Dense(
			units = Intermediate_dims,
			activation = tf.nn.relu,
			kernel_initializer = tf.keras.initializers.he_normal(),
			name = 'hidden_layer'
		)
		self.output_layer = tf.keras.layers.Dense(
			units = Intermediate_dims,
			activation = tf.nn.sigmoid
		)
	
	def call(self, input_features):
		activation = self.hidden_layer(input_features)
		return self.output_layer(activation)


class Decoder(tf.keras.layers.Layer):
	def __init__(self, intermediate_dims, original_dim):
		super(Decoder,self).__init__()
		self.hidden_layer = tf.keras.layers.Dense(
			units = intermediate_dims,
			activation = tf.nn.relu,
			kernel_initializer = tf.keras.initializers.he_normal(),
		)
		self.output_layer = tf.keras.layers.Dense(
			units = original_dim, ## this Units Is Responsible For the output, Later we Will reshape the output
			activation = tf.nn.sigmoid
		)

	def call(self,input_features):
		activation = self.hidden_layer(input_features)
		return self.output_layer(activation)
	

class AutoEncoder(tf.keras.layers.Layer):
	def __init__(self, intermediate_dims, original_dim):
		super(AutoEncoder, self).__init__()
		self.encoder = Encoder(intermediate_dims)
		self.decoder = Decoder(intermediate_dims, original_dim)

	def call(self, input_features):
		z = self.encoder(input_features)
		reconstructed_features = self.decoder(z)
		return reconstructed_features

def loss(ModelData , originalData):
	reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(ModelData(originalData) , originalData)))
	return reconstruction_error

def train(loss, modelData, optimizer ,originalData ):
	with tf.GradientTape() as tape:
		generated_loss = loss(modelData,originalData)
		loss_value = tape.gradient(generated_loss, modelData.trainable_variables)
		gradient_variables = zip(loss_value, modelData.trainable_variables)
		optimizer.apply_gradients(gradient_variables)

autoencoder = AutoEncoder(intermediate_dims=64, original_dim=784)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

(training_features, _ ), (test_features, _) = tf.keras.datasets.mnist.load_data()
training_features = training_features / 255.0
test_features = test_features / 255.0
training_features = training_features.reshape(training_features.shape[0],
						training_features.shape[1] * training_features.shape[2])
training_features = training_features.astype("float32")

training_dataset = tf.data.Dataset.from_tensor_slices(training_features)
training_dataset = training_dataset.batch(BATCH_SIZE).shuffle(training_features.shape[0]).prefetch(BATCH_SIZE * 4) 

writer = tf.summary.create_file_writer("tmp")

with writer.as_default():
	with tf.summary.record_if(True):
		for epoch in range(EPOCHS):
			print(epoch)
			for step, batch_features in enumerate(training_dataset):
				train(loss,autoencoder, optimizer,batch_features)
				loss_values = loss(autoencoder, batch_features)
				original = tf.reshape(batch_features, (batch_features.shape[0], 28, 28 ,1))
				theProposedInput = autoencoder(batch_features)
				reconstructed = tf.reshape(theProposedInput, (batch_features.shape[0], 28, 28, 1))
				tf.summary.scalar("Loss", loss_values, step=step)
				tf.summary.image("Original", original, step=step)
				tf.summary.image("reconstructed",reconstructed,max_outputs=10,step=step)

				

0
1
2
3
4


In [16]:
%tensorboard --logdir tmp/

UsageError: Line magic function `%tensorboard` not found.
